# Lecture 4

# Tool Calling, function calling

### Note: This is one of the most concept that that is as of today you can add business logic with just a python function and let llm use it.

Refer this: https://python.langchain.com/v0.1/docs/modules/model_io/chat/function_calling/

In [1]:
%pip install -qU langchain-google-genai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file

# Corrected way to retrieve the API key
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    raise ValueError("GOOGLE_API_KEY not found. Make sure it's set in the .env file.")

os.environ["GOOGLE_API_KEY"] = api_key

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

## Understanding Tool Calling

In [3]:
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.tools import tool

### Define Tool

In [4]:
@tool
def bmi_calculator(weight: float, height: float) -> str:
    """
    Calculate the BMI of a person given their weight and height.
    """

    bmi = weight/(height**2)

    if bmi < 18.5:
        return "Underweight"
    elif bmi >= 18.5 and bmi < 24.9:
        return "Normal weight"
    elif bmi >= 24.9 and bmi <29.9:
        return "Overweight"
    else:
        return "Obesity"
    
tools = [bmi_calculator]

### Bind your LLM with tools

In [5]:
llm_with_tool = llm.bind_tools(tools)

In [6]:
query = "What is the BMI of a person who is 100kg, and 1.75m tall?"

In [7]:
message = [HumanMessage(content=query)]
ai_message = llm_with_tool.invoke(message)
message

[HumanMessage(content='What is the BMI of a person who is 100kg, and 1.75m tall?', additional_kwargs={}, response_metadata={})]

In [8]:
ai_message

AIMessage(content='', additional_kwargs={'function_call': {'name': 'bmi_calculator', 'arguments': '{"height": 1.75, "weight": 100.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-9d5030c3-6171-4fbe-9cf5-a45fb0b0f148-0', tool_calls=[{'name': 'bmi_calculator', 'args': {'height': 1.75, 'weight': 100.0}, 'id': 'a635d263-5458-4a96-b0f2-17d934f0f075', 'type': 'tool_call'}], usage_metadata={'input_tokens': 43, 'output_tokens': 5, 'total_tokens': 48, 'input_token_details': {'cache_read': 0}})

In [9]:
message.append(ai_message)

print(message)

[HumanMessage(content='What is the BMI of a person who is 100kg, and 1.75m tall?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'bmi_calculator', 'arguments': '{"height": 1.75, "weight": 100.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-9d5030c3-6171-4fbe-9cf5-a45fb0b0f148-0', tool_calls=[{'name': 'bmi_calculator', 'args': {'height': 1.75, 'weight': 100.0}, 'id': 'a635d263-5458-4a96-b0f2-17d934f0f075', 'type': 'tool_call'}], usage_metadata={'input_tokens': 43, 'output_tokens': 5, 'total_tokens': 48, 'input_token_details': {'cache_read': 0}})]


In [10]:
for i in ai_message.tool_calls:
    print(i)

{'name': 'bmi_calculator', 'args': {'height': 1.75, 'weight': 100.0}, 'id': 'a635d263-5458-4a96-b0f2-17d934f0f075', 'type': 'tool_call'}


In [12]:
for i in ai_message.tool_calls:
    selected_tool = {"bmi_calculator": bmi_calculator}[i['name'].lower()]
    print(selected_tool)

name='bmi_calculator' description='Calculate the BMI of a person given their weight and height.' args_schema=<class 'langchain_core.utils.pydantic.bmi_calculator'> func=<function bmi_calculator at 0x0000018F1F3C5940>


In [13]:
for i in ai_message.tool_calls:
    selected_tool = {"bmi_calculator": bmi_calculator}[i['name'].lower()]
    print(selected_tool)
    tool_output = selected_tool.invoke(i['args'])
    print(tool_output)
    message.append(ToolMessage(content=tool_output, tool_call_id=i['id']))

print(message)

name='bmi_calculator' description='Calculate the BMI of a person given their weight and height.' args_schema=<class 'langchain_core.utils.pydantic.bmi_calculator'> func=<function bmi_calculator at 0x0000018F1F3C5940>
Obesity
[HumanMessage(content='What is the BMI of a person who is 100kg, and 1.75m tall?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'bmi_calculator', 'arguments': '{"height": 1.75, "weight": 100.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-9d5030c3-6171-4fbe-9cf5-a45fb0b0f148-0', tool_calls=[{'name': 'bmi_calculator', 'args': {'height': 1.75, 'weight': 100.0}, 'id': 'a635d263-5458-4a96-b0f2-17d934f0f075', 'type': 'tool_call'}], usage_metadata={'input_tokens': 43, 'output_tokens': 5, 'total_tokens': 48, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Obesity', tool_call_id='a635d263-5458-4a

In [19]:
llm_with_tool.invoke(message).content

'The BMI calculator indicates that a person who weighs 100kg and is 1.75m tall is obese.  However, the provided `bmi_calculator` function is incomplete and does not actually return a BMI value.  It only returns a string indicating the weight class.  To get the actual BMI value, the function needs to be modified to perform the calculation.'